# Zernike Polynomial Evaluation

This notebook compares different methods for evaluating the radial part of the Zernike polynomials, in terms of both speed and accuracy

The two primary methods we consider are direct polynomial evaluation using Horner's method and an evaluation scheme based on a recurrence relation for Jacobi polynomials.

The radial part of the Zernike polynomials is given by

$$
    \mathcal{R}_l^{|m|} (\rho) = \sum_{s=0}^{(l-|m|)/2} \frac{(-1)^s(l-s)!}{ s!\left( \cfrac{l+|m|}{2} - s\right)! \left( \cfrac{l-|m|}{2} - s\right)!  } \rho^{l-2s}
$$

Because the coefficient of rho is made up of entirely integer operations, it can be evaluated quickly and exactly to arbitrary orders (recall that python natively supports arbitrary length integer arithmetic). These coefficients can then be evaluated using Horner's method. This is done in the `zernike_radial_poly` function.

The other approach uses the fact that the above equation can be written as

$$
    \mathcal{R}_l^{m} (\rho) = (-1)^{(l-m)/2} \rho^m  P_{(l-m)/2}^{m, 0} (1 - 2 \rho^2)  \hspace{1cm}\text{for } m\geq0
$$
where $P_{n}^{\alpha, \beta}(\rho)$ is a Jacobi polynomial. This allows us to use stable recurrence relations for the Jacobi polynomials, as is done in the `zernike_radial` function.

The recurrence relationship for the Jacobi polynomials is,
$$
    2n(c-n)(c-2)P_{n}^{\alpha,\beta}(\rho) = (c-1)[c(c-2)\rho + (a-b)(c-2n)]P_{n-1}^{\alpha,\beta}(\rho) - 2(a-1)(b-1)cP_{n-2}^{\alpha,\beta}(\rho)
$$
where 
$$
    c = 2n + \alpha + \beta, \hspace{1cm} a = n +\alpha, \hspace{1cm} b = n + \beta
$$

For the derivatives of Zernike Radial part, we will also need derivatives of Jacobi polynomials, for which there exist another relation. 
$$
    \cfrac{d^k}{dx^k} P_n^{(\alpha, \beta)}(x) = \cfrac{\Gamma(\alpha + \beta + n + 1 + k)}{2^k\Gamma(\alpha + \beta + n + 1)} P_{n-k}^{(\alpha + k, \beta + k)}(x)
$$
This function relates the derivatives to normal Jacobi function, and we can use above recursion relation to calculate derivatives. To further reduce the numerical inaccuracies, we can use Pochammer form instead of Gamma function. This gives us,
$$
    \cfrac{d^k}{dx^k} P_n^{(\alpha, \beta)}(x) = \cfrac{(\alpha + \beta + n + 1)_k}{2^k} P_{n-k}^{(\alpha + k, \beta + k)}(x)
$$
where
$$
    (\alpha + \beta + n + 1)_k = (\alpha + \beta + n + 1)(\alpha + \beta + n + 2) ... (\alpha + \beta + n + k)
$$

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

In [ ]:
import numpy as np
import mpmath
import matplotlib
import matplotlib.pyplot as plt
from zernipax.basis import ZernikePolynomial
from zernipax.zernike import(
    polyder_vec,
    zernike_radial,
    zernike_radial_old_desc,
    zernike_radial_coeffs,
    zernike_radial_poly,
)
from zernipax.plotting import plot_comparison

In [ ]:
res = 50
basis = ZernikePolynomial(L=res, M=res, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)

Here we time the evaluation for a basis set containing 676 modes on a grid of 100 points, for derivative orders 0 through 3. (note the `block_until_ready` is needed to get [accurate timing with jax](https://jax.readthedocs.io/en/latest/async_dispatch.html))

In [ ]:
print("zernike_radial, 0th derivative")
%timeit _ = zernike_radial(r, basis.modes[:,0], basis.modes[:,1], 0).block_until_ready()
print("zernike_radial, 1st derivative")
%timeit _ = zernike_radial(r, basis.modes[:,0], basis.modes[:,1], 1).block_until_ready()
print("zernike_radial, 2nd derivative")
%timeit _ = zernike_radial(r, basis.modes[:,0], basis.modes[:,1], 2).block_until_ready()
print("zernike_radial, 3rd derivative")
%timeit _ = zernike_radial(r, basis.modes[:,0], basis.modes[:,1], 3).block_until_ready()

In [ ]:
print("zernike_radial_poly, 0th derivative")
%timeit _ = zernike_radial_poly(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], dr=0, exact=False)
print("zernike_radial_poly, 1st derivative")
%timeit _ = zernike_radial_poly(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], dr=1, exact=False)
print("zernike_radial_poly, 2nd derivative")
%timeit _ = zernike_radial_poly(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], dr=2, exact=False)
print("zernike_radial_poly, 3rd derivative")
%timeit _ = zernike_radial_poly(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], dr=3, exact=False)

We see that the implementation using Jacobi polynomial recurrence relation is significantly faster, despite the overhead from the JAX just-in-time compiler

For accuracy comparison, we will also evaluate the Zernike radial polynomials in extended precision (100 digits of accuracy) and treat this as the "true" value.

In [ ]:
mpmath.mp.dps = 100
print("Calculate radial Zernike polynomial coefficients (exact)")
%time c = zernike_radial_coeffs(basis.modes[:, 0], basis.modes[:, 1], exact=True)

print("\nzernike_radial_exact, 0th derivative")
%time zt0 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in c]).T
print("zernike_radial_exact, 1st derivative")
%time zt1 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in polyder_vec(c, 1, exact=True)]).T
print("zernike_radial_exact, 2nd derivative")
%time zt2 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in polyder_vec(c, 2, exact=True)]).T
print("zernike_radial_exact, 3rd derivative")
%time zt3 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in polyder_vec(c, 3, exact=True)]).T

Next we can plot the error resulting from the two evaluation methods (polynomial evaluation and jacobi recurrence relation) vs the true solution computed in exact arithmetic. We plot the max absolute error as well as the max relative error over $\rho \in (0,1)$ for each derivative order.

In [ ]:
mpmath.mp.dps = 100
c = zernike_radial_coeffs(basis.modes[:, 0], basis.modes[:, 1], exact=True)
zt0 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in c]).T

zr0 = zernike_radial(r, basis.modes[:, 0], basis.modes[:, 1], 0)
zr1 = zernike_radial(r, basis.modes[:, 0], basis.modes[:, 1], 1)
zr2 = zernike_radial(r, basis.modes[:, 0], basis.modes[:, 1], 2)
zr3 = zernike_radial(r, basis.modes[:, 0], basis.modes[:, 1], 3)
zp0 = zernike_radial_poly(
    r[:, np.newaxis], basis.modes[:, 0], basis.modes[:, 1], dr=0, exact=False
)
zp1 = zernike_radial_poly(
    r[:, np.newaxis], basis.modes[:, 0], basis.modes[:, 1], dr=1, exact=False
)
zp2 = zernike_radial_poly(
    r[:, np.newaxis], basis.modes[:, 0], basis.modes[:, 1], dr=2, exact=False
)
zp3 = zernike_radial_poly(
    r[:, np.newaxis], basis.modes[:, 0], basis.modes[:, 1], dr=3, exact=False
)

## Absolute Error

### 0th derivative

In [ ]:
plot_comparison(zt0, (zp0, zr0), basis, 0, "absolute", names=("Poly: ", "Radial: "), print_error=True)
plt.savefig("compare.png", dpi=1000)

### 1st derivative

In [ ]:
plot_comparison(zt1, (zp1, zr1), basis, 1, "absolute")

### 2nd derivative

In [ ]:
plot_comparison(zt2, (zp2, zr2), basis, 2, "absolute")

### 3rd derivative

In [ ]:
plot_comparison(zt3, (zp3, zr3), basis, 3, "absolute")

## Relative Error

### 0th derivative

In [ ]:
plot_comparison(zt0, (zp0, zr0), basis, 0, "relative")

### 1st derivative

In [ ]:
plot_comparison(zt1, (zp1, zr1), basis, 1, "relative")

### 2nd derivative

In [ ]:
plot_comparison(zt2, (zp2, zr2), basis, 2, "relative")

### 3rd derivative

In [ ]:
plot_comparison(zt3, (zp3, zr3), basis, 3, "relative")

So in addition to being faster, the evaluation using the Jacobi recurrence relation is also significantly more accurate as the mode numbers increase, keeping absolute error less than $10^{-5}$ and relative error less than $10^{-9}$, while directly evaluating the polynomial leads to errors greater than 100\% for large $l$